Final tests for the journal review. What is tested:
    
    - get/plot more params (FLOPS, model size etc.)
    - PCD vis: GT vs Pred
    - Test for all noises combined
    

In [2]:
# Imports:
import time
import imports
import torch
import os
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DenseDataLoader
from torch_geometric.transforms import FixedPoints, Compose, NormalizeScale, NormalizeRotation, RandomRotate
from FilteredShapenetDataset import FilteredShapeNet, ShapeNetCustom
from RGCNNSegmentation import seg_model
import numpy as np

import pickle

from pathlib import Path
from collections import defaultdict

from utils import BoundingBoxRotate, label_to_cat
from utils import seg_classes

from TestModel import ModelTester

/home/rocon/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [3]:
# Load the dataset 
dataset_name = "Original_2048"
dataset = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", split="test", transform=FixedPoints(2048), categories=seg_classes)
print(dataset[0])

Data(x=[2048, 3], y=[2048], pos=[2048, 3], category=[1])


In [5]:
path = "final_models"
model_names = ["2048_seg_clean.pt", "2048_seg_bb.pt", "2048_seg_rrbb.pt", "2048_seg_eig.pt", "2048_seg_gauss_rr_bb.pt", "2048_seg_gauss_rr_eig.pt"]
model_names = [path + "/" + model for model in model_names] 
num_points = 2048
input_dim  = 22

F = [128, 512, 1024]  # Outputs size of convolutional filter.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128, 50]

for model in model_names:
    net = seg_model(num_points, F, K, M, input_dim, dropout=0.2, reg_prior=False)
    net.load_state_dict(torch.load(f"{imports.curr_path}/{model}"))
    net.eval()
    print(f"\nModel: {model}")
    print(get_n_params(net))
    print(net)
    


Model: final_models/2048_seg_clean.pt
7413666
seg_model(
  (dropout): Dropout(p=0.2, inplace=False)
  (bias_relus): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 1x2048x128]
      (1): Parameter containing: [torch.FloatTensor of size 1x2048x512]
      (2): Parameter containing: [torch.FloatTensor of size 1x2048x1024]
      (3): Parameter containing: [torch.FloatTensor of size 1x2048x512]
      (4): Parameter containing: [torch.FloatTensor of size 1x2048x128]
      (5): Parameter containing: [torch.FloatTensor of size 1x2048x50]
  )
  (conv): ModuleList(
    (0): DenseChebConvV2(in_features=22, out_features=128, K=6, bias=True)
    (1): DenseChebConvV2(in_features=128, out_features=512, K=5, bias=True)
    (2): DenseChebConvV2(in_features=512, out_features=1024, K=3, bias=True)
  )
  (batch_norm_list_conv): ModuleList(
    (0): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, a

In [9]:
# Test FLOPS counter:
from fvcore.nn import FlopCountAnalysis

with torch.no_grad():
    class NetWrapper(torch.nn.Module):
        def __init__(self, net):
            super(NetWrapper, self).__init__()
            self.net = net
        
        def forward(self, data):
            return self.net(data[0], data[1])
        

    model = "/final_models/2048_seg_clean.pt"
    pos = dataset[0].pos.unsqueeze(0)
    norm = dataset[0].x.unsqueeze(0)
    x = torch.cat([pos.type(torch.float32), norm.type(torch.float32)], dim=2)
    cat = dataset[0].category
    net = seg_model(2048, F, K, M, input_dim, dropout=0.2, reg_prior=False)
    print(f"{imports.curr_path}/{model}")
    net.load_state_dict(torch.load(f"{imports.curr_path}/{model}"))
    net = net.eval()
    out, x , _ = net(x, cat)
    print(out.shape)
    # net_wrapped = NetWrapper(net)


    # data = [x.detach(), cat.detach()]

    # flops = FlopCountAnalysis(net_wrapped, data)
    # print(f"Total flops:       {flops.total()}")
    # print(f"Flops by operator: {flops.by_operator()}")
    # print(f"Flops by module:   {flops.by_module()}")


/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode//final_models/2048_seg_clean.pt
torch.Size([1, 2048, 50])
[]


In [1]:
# Imports:
import time
import imports
import torch
import os
from torch_geometric.datasets import ShapeNet
from torch_geometric.loader import DenseDataLoader
from torch_geometric.transforms import FixedPoints, Compose, NormalizeScale, NormalizeRotation, RandomRotate
from FilteredShapenetDataset import FilteredShapeNet, ShapeNetCustom
from RGCNNSegmentation import seg_model
import numpy as np

import pickle

from pathlib import Path
from collections import defaultdict

from utils import BoundingBoxRotate, label_to_cat
from utils import seg_classes

from TestModel import ModelTester
model_path = "/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode/final_models/2048_seg_clean.pt"

num_points = 2048
input_dim  = 22

F = [128, 512, 1024]  # Outputs size of convolutional filter.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128]

model = seg_model(num_points, F, K, M, input_dim, dropout=0.2, reg_prior=False)
model.load_state_dict(torch.load(model_path))
model.eval()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


RuntimeError: Error(s) in loading state_dict for seg_model:
	Unexpected key(s) in state_dict: "bias_relus.5", "batch_norm_list_fc.2.weight", "batch_norm_list_fc.2.bias", "batch_norm_list_fc.2.running_mean", "batch_norm_list_fc.2.running_var", "batch_norm_list_fc.2.num_batches_tracked", "fc.2.weight", "fc.2.bias". 

Point 2

- GT no rot
- GT rot
- RGCNN no rot
- RGCNN rot
- Ours no rot
- Ours rot

Also thest DGCNN and PointNet: 

- https://github.com/antao97/dgcnn.pytorch/blob/master/main_cls.py


Time complexity:

- Eig - Alex GIT (/ NormalizeRot Pytorch???)
- GRAMM + Model - model dim increases
- BB
- multiview BB - model forward
- multiview Eig - model forward

Breakdown and time the modules of the alg

Noisy dataset visualization

[PointCloud with 2048 points., PointCloud with 2048 points., PointCloud with 2048 points., PointCloud with 2048 points.]


In [5]:
import imports
from RGCNNClassification import cls_model
num_points = [512, 1024, 2048]


F = [128, 512, 1024]  # Outputs size of convolutional filter.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128, 50]
input_dim = 6


for num in num_points:
    transform = FixedPoints(num)
    dataset = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=transform, categories=seg_classes)
    pcd = dataset[0]
    pos = pcd.pos.unsqueeze(0)
    norm = pcd.x.unsqueeze(0)
    x = torch.cat([pos.type(torch.float32), norm.type(torch.float32)], dim=2)
    cat = pcd.category
    
    x = x.to("cuda")
    cat = cat.to("cuda")
    net = cls_model(num, F, K, M, input_dim, dropout=0.2, reg_prior=False)
    net = net.eval()
    net = net.to('cuda')
    
    start = time.time()
    for _ in range(1000):
        a, b, c = net(x)
    stop = time.time()
    
    print(f"Num Points: {num} \t-- {(stop-start)/1000}")

Num Points: 512 	-- 0.004919095039367676
Num Points: 1024 	-- 0.012095909833908082
Num Points: 2048 	-- 0.05739609980583191


In [4]:
num_points = [512, 1024, 2048]


F = [128, 512, 1024]  # Outputs size of convolutional filter.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128, 50]
input_dim = 22


for num in num_points:
    transform = FixedPoints(num)
    dataset = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=transform, categories=seg_classes)
    pcd = dataset[0]
    pos = pcd.pos.unsqueeze(0)
    norm = pcd.x.unsqueeze(0)
    x = torch.cat([pos.type(torch.float32), norm.type(torch.float32)], dim=2)
    cat = pcd.category
    
    x = x.to("cuda")
    cat = cat.to("cuda")
    net = seg_model(num, F, K, M, input_dim, dropout=0.2, reg_prior=True)
    net = net.eval()
    net = net.to('cuda')
    
    start = time.time()
    for _ in range(1000):
        a, b, c = net(x, cat)
    stop = time.time()
    
    print(f"Num Points: {num} \t-- {(stop-start)/1000}")
    
    

Num Points: 512 	-- 0.007277761220932007
Num Points: 1024 	-- 0.015940637350082398
Num Points: 2048 	-- 0.06178945708274841


In [1]:
import open3d as o3d
import imports
# from noise_visualization import get_noisy_pcd, show_pcds
import torch_geometric
from torch_geometric.datasets import ShapeNet
from torch_geometric.transforms import NormalizeScale, Compose, FixedPoints, RandomRotate
import numpy as np
from utils import seg_classes


class VisualizerPCD():
    def __init__(self) -> None:
        self.pcds_o3d = []        
        self.colors = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1],[0.3, 0.4, 0.6]])
        # self.colors = np.random.rand(51, 3)
            
    def _prepare_pcds(self, pcds, y, row_nr=0, col_number=0):
        
        if not type(pcds) == list:
            pcds = [pcds]
            y = [y]
        
        pcds_o3d = [None] * len(pcds)
        for i, pcd in enumerate(pcds):
            pcd_o3d = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd))
            pcd_o3d.colors = o3d.utility.Vector3dVector(self.colors[y[i]]) 
            pcd_o3d = pcd_o3d.translate([col_number*2.5, row_nr*1.5, 0])
            pcds_o3d[i] = pcd_o3d        
        return pcds_o3d
    
    
    def add(self, pcds, row_nr, col_nr=0):
        if not type(pcds)==list:
            pcds = [pcds]
        for i, pcd in enumerate(pcds):
            pos = pcd.pos
            y = pcd.y - min(pcd.y)
            self.add_pcds(pos, y, row_nr+i, col_nr)
    
    
    def add_pcds(self, pcds, y, row_nr, col_nr=0):
        self.pcds_o3d += self._prepare_pcds(pcds, y, row_nr=row_nr, col_number=col_nr)

        
    def show(self):
        o3d.visualization.draw_geometries(self.pcds_o3d)


vis = VisualizerPCD()


# dataset_name = "Original_2048"
transform = Compose([NormalizeScale(), FixedPoints(2048)])
dataset = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=transform, categories=seg_classes)
print(imports.dataset_path)
pcd1 = dataset[1]
pcd2 = dataset[2]
pcd3 = [pcd1, pcd2]
vis.add(pcd1, 0, 0)
vis.add(pcd2, 1, 0)
vis.add(pcd3, 0, 1)
vis.show()

/home/rocon/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/rocon/Workspace/rgcnn_pytorch/dataset


: 

: 

In [62]:
d_custom = ShapeNetCustom(root_dir=Path("/home/domsa/workspace/git/rgcnn_pytorch/dataset/Journal/ShapeNetCustom/Original_2048"), folder="test", transform=FixedPoints(2048))
print(len(d_custom))
pcustom = d_custom[2873]
print(pcustom)

vis = VisualizerPCD()
vis.add(pcustom, 0, 0)
vis.show()

2874
Data(x=[2048, 3], y=[2048], pos=[2048, 3], category=[1])
[PointCloud with 2048 points.]


In [20]:
from utils import GaussianNoiseTransform
from utils import Sphere_Occlusion_Transform
path = "final_models"
# model_names = ["2048_seg_clean.pt", "2048_seg_rrbb.pt", "2048_seg_eig.pt", "2048_seg_gauss_rr_bb.pt", "2048_seg_gauss_rr_eig.pt"]
model_names = ["2048_seg_clean.pt", "2048_seg_rrbb.pt", "2048_seg_gauss_rr_bb.pt", "2048_seg_gauss_rr_eig.pt",  "2048_seg_eig.pt"]
model_names = [path + "/" + model for model in model_names] 
num_points = 2048
input_dim  = 22


F = [128, 512, 1024]  # Outputs size of convolutional filter.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128, 50]


fp = FixedPoints(2048)
ns = NormalizeScale()
nr = NormalizeRotation()
t_n = Compose([fp, ns])
t_r = Compose([fp, ns, RandomRotate(90, 0), RandomRotate(90, 1), RandomRotate(90, 2)])
t_g = Compose([fp, ns, GaussianNoiseTransform(mu=0, sigma=0.02)])
t_s = Compose([fp, ns, Sphere_Occlusion_Transform(0.25, num_points=2048)])

rect_transform = [
    Compose([]),
    Compose([BoundingBoxRotate()]),
    Compose([BoundingBoxRotate()]),
    Compose([NormalizeRotation()]),
    Compose([NormalizeRotation()])
]


In [21]:
import copy

d_o = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=t_n, categories=seg_classes)
d_r = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=t_r, categories=seg_classes)
d_g = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=t_g, categories=seg_classes)
d_s = ShapeNet(root=f"{imports.dataset_path}/Journal/ShapeNet/", transform=t_s, categories=seg_classes)

po = d_o[1]
pr = d_r[1]
pg = d_g[1]
ps = d_s[1]

vis = VisualizerPCD()

input_pcds = [ps, pg, pr, po]

vis.add(input_pcds, 0, 0)

net = seg_model(2048, F, K, M, input_dim, reg_prior=False)

device = "cuda" if torch.cuda.is_available() else "cpu"

for k, model_name in enumerate(model_names):
    print(f"{imports.curr_path}/{model_name}")
    net.load_state_dict(torch.load(f"{imports.curr_path}/{model_name}"))
    net = net.eval()
    net = net.to(device)
    for i, pcd in enumerate(input_pcds):
        init_pos = pcd.pos
        pcd_in = copy.deepcopy(pcd)
        pcd_in = rect_transform[k](pcd_in)
        pos = pcd_in.pos.unsqueeze(0)
        norm = pcd_in.x.unsqueeze(0)
        x = torch.cat([pos.type(torch.float32), norm.type(torch.float32)], dim=2)
        # cat = pcd.category
        cat = torch.tensor(0)
        x, cat = x.to(device), cat.to(device)
        out, _ , _ = net(x, cat)
        y = out.argmax(dim=2)
        vis.add_pcds(init_pos, y.squeeze(0).detach().cpu(), i, k+1)

vis.show()

/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode/final_models/2048_seg_clean.pt
/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode/final_models/2048_seg_rrbb.pt
/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode/final_models/2048_seg_gauss_rr_bb.pt
/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode/final_models/2048_seg_gauss_rr_eig.pt
/home/domsa/workspace/git/rgcnn_pytorch/JurnalCode/final_models/2048_seg_eig.pt
